<a href="https://colab.research.google.com/github/priyansh4320/Abstractive-Text-Summarization-Enhancing-Sequence-to-Sequence-Models-Using-Word-Sense-Disambiguatio/blob/main/Attention_Based_seq2se1_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **deep learning model**
# ***Attention- based Seq2Seq Model***

# bidirectional LSTM encoder :phase 1

In [ ]:
import pandas as  pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Bidirectional
from keras.models import Sequential

In [ ]:
from keras.layers import *
from keras.models import *
from keras import backend as K

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/ATS- gigadata/model_gigadata_new.csv")

In [ ]:
i=0
tokens ,art_summer=[] , []

while i<1000000:
  art_summer.append(df_train['article'][i])
  art_summer.append(df_train['summery'][i])
  tokens.append(art_summer)
  art_summer = []
  i+=1

In [ ]:
embed_model = Word2Vec(tokens, size = 100, window = 5 , min_count = 1)

In [ ]:
embed_model.save("w2v.model")

In [ ]:
embedding_weights = embed_model.wv.vectors

In [ ]:
embedding_weights.shape

(1498136, 100)

In [ ]:
x , y = [],[]
i=0
while i<len(df_train):
  x.append(df_train['article'][i])
  y.append(df_train['summery'][i])
  i+=1

In [ ]:
len(x)

1000000

In [ ]:
# Tokenize the input text
tokenizer = Tokenizer()
text_data = tokens  #input data
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index
word_index

{'gpe organization': 1,
 'gpe person': 2,
 'person person': 3,
 'person organization': 4,
 'gpe person # . # # person': 5,
 'gpe person # . # person': 6,
 'gpe organization in gpe': 7,
 'person person organization': 8,
 'gpe person organization': 9,
 'person person in gpe': 10,
 'person organization in gpe': 11,
 'gpe person in gpe': 12,
 'gpe person to person': 13,
 'person person to person': 14,
 'gpe organization # . # # person': 15,
 "gpe 's organization": 16,
 'gpe person for person': 17,
 'person person for person': 18,
 'gpe organization # . # person': 19,
 'gpe organization on person': 20,
 'gpe person on person': 21,
 'person person # . # person': 22,
 "gpe 's person": 23,
 'person person # . # # person': 24,
 'gpe person after person': 25,
 'gpe organization after person': 26,
 'person person with person': 27,
 'person person on person': 28,
 'gpe person as person': 29,
 'gpe person # . # # pc': 30,
 'person person after person': 31,
 'gpe person with person': 32,
 'gpe to pe

In [ ]:
#increasing length of word index to change the shape
i=1
while i<=10:
  word_index['person organization gpe ',i] = 1000+i
  i+=1

In [ ]:
type(word_index)

dict

In [ ]:
len(word_index)

1498135

In [ ]:
sorted(word_index.values())[-1]

1498125

In [ ]:
train_x = x
train_y = y

In [ ]:
# Convert words to word vectors
tokenizer.fit_on_texts(train_x)
sequences_x = tokenizer.texts_to_sequences(train_x)
padded_sequences_x = np.array(pad_sequences(sequences_x))
print(padded_sequences_x)


tokenizer.fit_on_texts(train_y)
sequences_y = tokenizer.texts_to_sequences(train_y)
padded_sequences_y = np.array(pad_sequences(sequences_y))
print(padded_sequences_x)

[[    0     0     0 ...     3    73    16]
 [    0     0     0 ...  2758    62     1]
 [    0     0     0 ... 29456     6     4]
 ...
 [    0     0     0 ...     2    51    23]
 [    0     0     0 ...     5   157    39]
 [    0     0     0 ...    69     9   172]]
[[    0     0     0 ...     3    73    16]
 [    0     0     0 ...  2758    62     1]
 [    0     0     0 ... 29456     6     4]
 ...
 [    0     0     0 ...     2    51    23]
 [    0     0     0 ...     5   157    39]
 [    0     0     0 ...    69     9   172]]


In [ ]:
padded_sequences_x.shape

(1000000, 79)

In [ ]:
padded_sequences_y.shape

(1000000, 38)

In [ ]:
len(word_index)

1498125

In [ ]:
from keras.layers.rnn.time_distributed import Layer
class attention(Layer):

  def __init__(self, return_sequences=True):
      self.return_sequences = return_sequences

      super(attention,self).__init__()

  def build(self, input_shape):
      self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1), initializer="normal")
      self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),initializer="normal")
      self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),initializer="normal")
      self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1), initializer="normal")

      super(attention,self).build(input_shape)


  def call(self, x):
      e = K.tanh(K.dot(x,self.W)+self.b)
      a = K.softmax(e, axis=1)
      output = x*a
      if self.return_sequences:
        return output

      return K.sum(output, axis=1)

In [ ]:
# Define  model
model = Sequential()
model.add(Embedding(len(word_index)+1 ,
                    embedding_weights.shape[1],
                    weights=[embedding_weights],
                    input_length=padded_sequences_x.shape[1],
                    trainable=False))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(attention(return_sequences=False)) # receive 3D and output 3D
model.add(RepeatVector(len(padded_sequences_y[1]))) # repeat vector
model.add(LSTM(32, return_sequences=True))  #decoder layer
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(1, activation='softmax')))
#model.add(Dense(1, activation='softmax'))


In [ ]:
# Train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 79, 100)           149813600 
                                                                 
 bidirectional (Bidirectiona  (None, 79, 64)           34048     
 l)                                                              
                                                                 
 attention (attention)       (None, 64)                143       
                                                                 
 repeat_vector (RepeatVector  (None, 38, 64)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 38, 32)            12416     
                                                                 
 dropout (Dropout)           (None, 38, 32)            0

In [ ]:
print(padded_sequences_x.shape , padded_sequences_y.shape)

(1000000, 79) (1000000, 38)


In [ ]:
tf.version

<module 'tensorflow.compat.v2.version' from '/usr/local/lib/python3.8/dist-packages/tensorflow/_api/v2/compat/v2/version/__init__.py'>

In [ ]:
model.fit(padded_sequences_x, padded_sequences_y, batch_size=64, epochs=15)